<a href="https://colab.research.google.com/github/nickohl/DELTA_SU/blob/main/code/img_lab_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#attach to Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
ls /content/drive/MyDrive/Thesis_Images/

img/  Label_Screenshot.png  S1_1033_label.tif
lab/  S1_1033_image.tif     UVSR_1034_Img.tif


In [ ]:
ls /content/drive/MyDrive/Thesis_Images/img

WV_1033.tif


In [ ]:
ls /content/drive/MyDrive/Thesis_Images/lab

WV_1033.tif


In [ ]:
# error check loop
# 1. confirm that there is a label for every image
# 2. confirm that they are in the same CRS
# 3. confirm that they overlap
# 4. confirm that they are of the same dimensions
# 5. confirm that there are no nan values
# 6. confirm that there are the same number of class values per image




import pandas as pd
import os
from osgeo import gdal
import numpy
# Change to your image directory
os.chdir('/content/drive/MyDrive/Thesis_Images/img')
check = []
files = glob.glob('*.tif')
#labs = glob.glob('/lab')
for file in files:
  lab_file = os.path.join('../lab', file)
  if os.path.exists(lab_file):
    labTF = True

    raster1 = gdal.Open(file)
    raster2 = gdal.Open(lab_file)
    
    # band1 = raster1.GetRasterBand(1)
    # band2 = raster2.GetRasterBand(1)
    gt1 = raster1.GetGeoTransform()
    gt2 = raster2.GetGeoTransform()
    
    # find each image's bounding box
    # r1 has left, top, right, bottom of dataset's bounds in geospatial coordinates.
    r1 = [gt1[0], gt1[3], gt1[0] + (gt1[1] * raster1.RasterXSize), gt1[3] + (gt1[5] * raster1.RasterYSize)]
    r2 = [gt2[0], gt2[3], gt2[0] + (gt2[1] * raster2.RasterXSize), gt2[3] + (gt2[5] * raster2.RasterYSize)]

    #r1 = numpy.array(ds1.ReadAsArray())
    #r2 = numpy.array(ds2.ReadAsArray())
    x_img = raster1.RasterXSize
    y_img = raster1.RasterYSize
    x_lab = raster2.RasterXSize
    y_lab = raster2.RasterYSize
    if r1 != r2:
      intersectTF = False
    else:
      intersectTF = True

    if x_img != x_lab | y_img != y_lab:
      samesizeTF = True
    else:
      samesizeTF = False
    nanTF_img = numpy.isnan(numpy.array(raster1.ReadAsArray())).any()
    nanTF_lab = numpy.isnan(numpy.array(raster1.ReadAsArray())).any()
    lablevs = numpy.unique(numpy.array(raster2.ReadAsArray()))
  else:
    labTF = False
    intersectTF = samesizeTF=nanTF_img = nanTF_lab = lablevs= numpy.nan
    r1 = r2 = [numpy.nan, numpy.nan, numpy.nan, numpy.nan]
    x_img = y_img = x_lab = y_lab = numpy.nan
check.append([file, labTF, intersectTF,r1, r2, samesizeTF, x_img, y_img, x_lab, y_lab, nanTF_img, nanTF_lab, lablevs])



In [ ]:
check

[['WV_1033.tif',
  True,
  False,
  [-94.89393136531177,
   30.08527183536001,
   -94.444095798487,
   29.820985835360013],
  [-94.89255172858091,
   30.07810000398323,
   -94.46566909201842,
   29.84061772117073],
  True,
  24401,
  14336,
  23332,
  12980,
  False,
  False,
  array([  0,   1, 255], dtype=uint16)]]